## CTA BIODIV ROUND 2

In [1]:
import pandas as pd
import os
import time
import json
import requests
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import Counter
from collections import OrderedDict
from ratelimit import limits, sleep_and_retry

In [2]:
## All path can be set according to your local repo
# Folder path containing ground truths
folder_path_ground_truth = "Dataset/tbiodiv10-0.01-sample/horizontal/gt/"

# Folder path given the target 
folder_path_tragets = "Dataset/tbiodiv10-0.01-sample/horizontal/targets/"

# Define folder path containing tables
folder_path_tables = "Dataset/tbiodiv10-0.01-sample/horizontal/tables/"

file_name_cta_gt = "cta_gt.csv"
file_name_cta_target = "cta_targets.csv"
file_name_specific_table = "AOD06100110I140.csv" # AZC06100207I0103, WIT06100910I067

file_path_cta_target = folder_path_tragets + file_name_cta_target
file_path_cta_gt = folder_path_ground_truth + file_name_cta_gt
file_path_specific_table = folder_path_tables + file_name_specific_table

#### Check the Target csv for the CTA task

In [3]:
## Check the target 
table_biodiv_cta_target = pd.read_csv(file_path_cta_target,header = None)

In [4]:
print(f"Shape of the target dataframe is: {table_biodiv_cta_target.shape}")
table_biodiv_cta_target.head()


Shape of the target dataframe is: (5250, 2)


,0,1
0,EGN060702I0010,1
1,EGN060702I0010,3
2,EGN060702I0031,1
3,EGN060702I0072,2
4,EGN060702I0114,2


In [5]:
# Check total unique tables in it having the first index containing table names.
print(f"Total unique table in given in the target CTA file {table_biodiv_cta_target[0].nunique()}")

# Check the duplicate rows in the csv file
duplicates = table_biodiv_cta_target.duplicated(subset=[table_biodiv_cta_target.columns[0], table_biodiv_cta_target.columns[1]])
print(f"Total duplicate row in the given target CTA files are: {duplicates.sum()}")

Total unique table in given in the target CTA file 1317
Total duplicate row in the given target CTA files are: 0


#### Check the Ground truth csv

In [6]:
table_biodiv_cta_gt = pd.read_csv(file_path_cta_gt, header=None)

In [7]:
print(f"Shape of the ground truth dataset is : {table_biodiv_cta_gt.shape}")
table_biodiv_cta_gt.head()

Shape of the ground truth dataset is : (5250, 3)


,0,1,2
0,EGN060702I0010,1,http://www.wikidata.org/entity/Q484170
1,EGN060702I0010,3,"http://www.wikidata.org/entity/Q5162954,http:/..."
2,EGN060702I0031,1,NIL
3,EGN060702I0072,2,http://www.wikidata.org/entity/Q16695773
4,EGN060702I0114,2,NIL


In [8]:
# Check total unique tables in it having the first index containing table names.
print(f"Total unique table in given in the ground truth CTA file {table_biodiv_cta_gt[0].nunique()}")

# Check the duplicate rows in the csv file
duplicates = table_biodiv_cta_gt.duplicated(subset=[table_biodiv_cta_gt.columns[0], table_biodiv_cta_gt.columns[1]])
print(f"Total duplicate row in the given ground truth CTA files are: {duplicates.sum()}")

Total unique table in given in the ground truth CTA file 1317
Total duplicate row in the given ground truth CTA files are: 0


In [9]:
### Finding common rows between target and ground samples 
## Get the first three column from both the table 
table_biodiv_cta_target_subset = table_biodiv_cta_target.iloc[:, :2]
table_biodiv_cta_gt_subset = table_biodiv_cta_gt.iloc[:, :2]

# Check whether all the rows are same or not
merged_df = pd.merge(table_biodiv_cta_target_subset, table_biodiv_cta_gt_subset, on=[0,1], how = 'inner')
print(f"The dimension of the merged dataframe is : {merged_df.shape}")

The dimension of the merged dataframe is : (5250, 2)


#### Add the column_values 

##### A faster way to fetch the specific column from the table and then store its values in the dataframe

In [10]:
# Function to read CSV files into a dictionary
def load_tables(folder_path_tables):
    tables = {}
    for filename in os.listdir(folder_path_tables):
        if filename.endswith('.csv'):
            table_name = filename[:-4]  # Remove '.csv' extension
            table_path = os.path.join(folder_path_tables, filename)
            tables[table_name] = pd.read_csv(table_path)
    return tables

def get_value_from_preloaded_tables(row, tables):
    table_name = row[0]
    column_number = row[1]

    if table_name in tables:
        df_table = tables[table_name]
        if column_number < len(df_table.columns):
            values = df_table.iloc[:, column_number]
            # Join the cell values with ' || ' as the separator
            return ' || '.join(values.astype(str).tolist())
    return None

# Function to fetch values in parallel
def fetch_values_in_parallel(df, tables):
    rows = df.itertuples(index=False)
    with ThreadPoolExecutor(max_workers=10) as executor:
        fetched_values = list(executor.map(lambda row: get_value_from_preloaded_tables(row, tables), rows))
    return fetched_values

In [11]:
%%time
# Load all tables into a dictionary
tables = load_tables(folder_path_tables)

CPU times: user 23.4 s, sys: 3.62 s, total: 27 s
Wall time: 1min 15s


In [13]:
%%time
# Fetch values in parallel
fetched_values = fetch_values_in_parallel(table_biodiv_cta_target, tables)

CPU times: user 2.29 s, sys: 637 ms, total: 2.93 s
Wall time: 2.92 s


In [14]:
# Add the fetched values as a new column in the dataframe
table_biodiv_cta_target['column_values'] = fetched_values

In [15]:
table_biodiv_cta_target.head()

,0,1,column_values
0,EGN060702I0010,1,Marchamp || Saint-Maurice-de-Gourdans || Nivig...
1,EGN060702I0010,3,"zone naturelle d'intérêt écologique, faunistiq..."
2,EGN060702I0031,1,Category:Judiciary of Iran || Category:Judicia...
3,EGN060702I0072,2,Wikipedia:Vital articles/Level/4 || Wikipedia:...
4,EGN060702I0114,2,Category:February 1910 events || Category:Febr...


In [16]:
def clean_column_values(cell_value):
    # Split the cell value by '||'
    groups = cell_value.split('||')
    cleaned_groups = []

    for group in groups:
        # Split each group by ',' and remove duplicates while preserving order
        elements = group.split(',')
        # Remove 'nan' values and strip whitespace
        elements = [element.strip() for element in elements if element.strip().lower() != 'nan']
        if elements:  # Only add non-empty groups
            unique_elements = ', '.join(OrderedDict.fromkeys(elements))
            cleaned_groups.append(unique_elements)
    
    # Remove duplicate groups while preserving order
    cleaned_groups = list(OrderedDict.fromkeys(cleaned_groups))

    # Join the cleaned groups back with '||'
    cleaned_value = ' || '.join(cleaned_groups)
    return cleaned_value

def clean_and_split_text(cell_value):
    # Split by '||' first
    groups = cell_value.split('||')
    cleaned_groups = []

    for group in groups:
        if group.strip():
            # Split each group by ',' followed by a space and a capital letter
            segments = re.split(r',\s+(?=[A-Z])', group)
            cleaned_segments = []
            
            for segment in segments:
                # Remove duplicates within the segment split by ','
                elements = segment.split(',')
                # Remove 'nan' values and strip whitespace
                elements = [element.strip() for element in elements if element.strip().lower() != 'nan']
                if elements:  # Only add non-empty segments
                    unique_elements = ', '.join(OrderedDict.fromkeys(elements))
                    cleaned_segments.append(unique_elements)
            
            # Join cleaned segments and maintain their order
            if cleaned_segments:
                cleaned_group = ' || '.join(cleaned_segments)
                cleaned_groups.append(cleaned_group)
        else:
            cleaned_groups.append(group.strip())
    
    # Join the cleaned groups back with '||'
    cleaned_value = ' || '.join(cleaned_groups)
    
    # Remove leading and trailing '||' if present
    cleaned_value = cleaned_value.strip(' ||')
    
    return cleaned_value

In [17]:
def combined_cleaning_function(cell_value):
    # First clean using clean_column_values
    cleaned_value = clean_column_values(cell_value)
    # Then further clean using clean_and_split_text
    fully_cleaned_value = clean_and_split_text(cleaned_value)
    return fully_cleaned_value

In [18]:
table_biodiv_cta_target['clean_column_values'] = table_biodiv_cta_target.iloc[:, 2].apply(combined_cleaning_function)

In [19]:
table_biodiv_cta_target.head()

,0,1,column_values,clean_column_values
0,EGN060702I0010,1,Marchamp || Saint-Maurice-de-Gourdans || Nivig...,Marchamp || Saint-Maurice-de-Gourdans || Nivig...
1,EGN060702I0010,3,"zone naturelle d'intérêt écologique, faunistiq...","zone naturelle d'intérêt écologique, faunistiq..."
2,EGN060702I0031,1,Category:Judiciary of Iran || Category:Judicia...,Category:Judiciary of Iran || Category:Judicia...
3,EGN060702I0072,2,Wikipedia:Vital articles/Level/4 || Wikipedia:...,Wikipedia:Vital articles/Level/4
4,EGN060702I0114,2,Category:February 1910 events || Category:Febr...,Category:February 1910 events || Category:Febr...


In [20]:
table_biodiv_cta_target.head()

,0,1,column_values,clean_column_values
0,EGN060702I0010,1,Marchamp || Saint-Maurice-de-Gourdans || Nivig...,Marchamp || Saint-Maurice-de-Gourdans || Nivig...
1,EGN060702I0010,3,"zone naturelle d'intérêt écologique, faunistiq...","zone naturelle d'intérêt écologique, faunistiq..."
2,EGN060702I0031,1,Category:Judiciary of Iran || Category:Judicia...,Category:Judiciary of Iran || Category:Judicia...
3,EGN060702I0072,2,Wikipedia:Vital articles/Level/4 || Wikipedia:...,Wikipedia:Vital articles/Level/4
4,EGN060702I0114,2,Category:February 1910 events || Category:Febr...,Category:February 1910 events || Category:Febr...


In [21]:
table_biodiv_cta_target_0_1000 = table_biodiv_cta_target[0:1000]
table_biodiv_cta_target_1000_2000 = table_biodiv_cta_target[1000:2000]
table_biodiv_cta_target_2000_3000 = table_biodiv_cta_target[2000:3000]
table_biodiv_cta_target_3000_4000 = table_biodiv_cta_target[3000:4000]
table_biodiv_cta_target_4000_5000 = table_biodiv_cta_target[4000:5000]
table_biodiv_cta_target_5000_5250 = table_biodiv_cta_target[5000:5250]

In [22]:
# Local cache to store results
wikidata_cache = {"id_cache": {}, "related_cache": {}}

# Define rate limit decorator to allow 10 requests per second
@sleep_and_retry
@limits(calls=10, period=1)
def rate_limited_request(url, params=None, headers=None):
    return requests.get(url, params=params, headers=headers)

def get_wikidata_id(label):
    if "id_cache" not in wikidata_cache:
        wikidata_cache["id_cache"] = {}
    if label in wikidata_cache["id_cache"]:
        return wikidata_cache["id_cache"][label]

    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "format": "json",
        "language": "en",
        "search": label.strip()
    }

    try:
        response = rate_limited_request(url, params=params)
        data = response.json()

        if data['search']:
            label_id = data['search'][0]['id']
            wikidata_cache["id_cache"][label] = label_id
            return label_id
        else:
            return None
    except (requests.JSONDecodeError, KeyError) as e:
        # print(f"Error: {e}")
        return None

def get_related_ids(label_id, property_id):
    if "related_cache" not in wikidata_cache:
        wikidata_cache["related_cache"] = {}
    cache_key = f"{label_id}_{property_id}"
    if cache_key in wikidata_cache["related_cache"]:
        return wikidata_cache["related_cache"][cache_key]

    url = "https://query.wikidata.org/sparql"
    query = f"""
    SELECT ?related WHERE {{
        wd:{label_id} wdt:{property_id} ?related.
    }}
    """
    
    headers = {
        "Accept": "application/sparql-results+json"
    }

    try:
        response = rate_limited_request(url, params={'query': query}, headers=headers)
        data = response.json()

        related_ids = []
        if data.get('results', {}).get('bindings'):
            for binding in data['results']['bindings']:
                related_uri = binding['related']['value']
                related_id = related_uri.split('/')[-1]
                related_ids.append(related_id)

        wikidata_cache["related_cache"][cache_key] = related_ids
        return related_ids
    except (requests.JSONDecodeError, KeyError) as e:
        # print(f"Error: {e}")
        return []

def process_cell(cell_contents):
    labels = [label.strip() for label in cell_contents.split('||')]
    filtered_labels = [label for label in labels if label.lower() != 'nan']
    unique_labels = list(set(filtered_labels))

    subclass_dict = {}
    for label in unique_labels:
        label_id = get_wikidata_id(label)
        if label_id:
            instance_ids = get_related_ids(label_id, 'P31')
            if instance_ids:
                subclass_dict[label] = instance_ids
            else:
                subclass_ids = get_related_ids(label_id, 'P279')
                subclass_dict[label] = subclass_ids if subclass_ids else [None]
        else:
            subclass_dict[label] = [None]

    subclass_ids_list = []
    for label in filtered_labels:
        subclass_ids_list.extend(subclass_dict[label])

    valid_subclass_ids = [subclass_id for subclass_id in subclass_ids_list if subclass_id is not None]

    if valid_subclass_ids:
        # Count occurrences of each ID
        counts = Counter(valid_subclass_ids)
        max_count = max(counts.values())
        # Select IDs that have the maximum count
        most_common_ids = [id for id, count in counts.items() if count == max_count]
        return ', '.join(most_common_ids)
    else:
        return None

# Save cache to file
def save_cache():
    with open('wikidata_cache.json', 'w') as cache_file:
        json.dump(wikidata_cache, cache_file)

# Load cache from file (when restarting script)
def load_cache():
    global wikidata_cache
    try:
        with open('wikidata_cache.json', 'r') as cache_file:
            wikidata_cache = json.load(cache_file)
            # Ensure cache structure is correct
            if "id_cache" not in wikidata_cache:
                wikidata_cache["id_cache"] = {}
            if "related_cache" not in wikidata_cache:
                wikidata_cache["related_cache"] = {}
    except (FileNotFoundError, json.JSONDecodeError):
        wikidata_cache = {"id_cache": {}, "related_cache": {}}

In [ ]:
%%time

# # Load the cache at the beginning of your notebook
load_cache()

# Now, let's process the DataFrame
# Iterate over rows of the DataFrame
for index, row in table_biodiv_cta_target_2000_3000.iterrows():
    cell_contents = row['clean_column_values']
    result = process_cell(cell_contents)
    table_biodiv_cta_target_2000_3000.at[index, 'column_type_annotation_multi_max_1'] = result

# # Save cache before script ends
save_cache()

<timed exec>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
table_biodiv_cta_target_2000_3000.head(20)

In [ ]:
table_biodiv_cta_target_2000_3000.to_csv("table_biodiv_cta_target_2000_3000_r2.csv", index=False)